# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 5
n_trials = 10
importance_type = "split"
#first round of optimization

life_history = ["lifespan", "mass_g", "mtGC", "metabolic_rate", "temperature", "gestation_days"]
trait = "lifespan"

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [7]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [8]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [9]:
default_selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
default_selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,[]


In [10]:
loader = DataLoader(locations, default_selection)
selections = loader.load_life_history()
selections[trait][0]

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 12323)","(12323, 37)",38,408,"(12323, 2)","(38, 18)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [11]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)

## Setting up features to select ##

In [12]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type = "split"
)

In [13]:
select_lifespan = replace(selection, select_by = "shap")

In [36]:
def load_study(trait: str):
    url = f'sqlite:///' +str((locations.interim.optimization / (trait+".sqlite")).absolute())
    print('loading (if exists) study from '+url)
    storage = optuna.storages.RDBStorage(
        url=url
        #engine_kwargs={'check_same_thread': False}
    )
    return optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], storage = storage, study_name = f"{trait}_r2_huber_kendall", load_if_exists = True)

study = load_study(trait)
metrics, params = MultiObjectiveResults.from_study(study).best_metrics_params_r2()
metrics, params

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/lifespan.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-24 20:41:03,873] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


((0.8895391594213731, 3.9388608314443507, 0.493533956976745),
 {'bagging_fraction': 0.9223944053685549,
  'boosting_type': 'gbdt',
  'drop_rate': 0.29847846764282016,
  'feature_fraction': 0.30501714460224083,
  'lambda_l1': 0.21894440516534,
  'lambda_l2': 1.3848191603958375,
  'learning_rate': 0.08210966706622723,
  'max_depth': 3,
  'max_leaves': 17,
  'min_data_in_leaf': 8,
  'objective': 'regression'})

In [39]:
len(study.get_pareto_front_trials())

8

In [30]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 3.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 3.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [31]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.R2, FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [32]:
results = p.fit_transform(selections[trait])
results

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


0
===== fitting models with seed 0 =====
SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Pan_paniscus']
Training until validation scores don't improve for 10 rounds
[250]	valid_0's huber: 8.70794	valid_0's l2: 303.514	valid_0's l1: 10.0568
Did not meet early stopping. Best iteration is:
[250]	valid_0's huber: 8.70794	valid_0's l2: 303.514	valid_0's l1: 10.0568
SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']
Training until validation scores don't improve for 10 rounds
[250]	valid_0's huber: 3.85972	valid_0's l2: 76.7909	valid_0's l1: 4.66369
Did not meet early stopping. Best iteration is:
[249]	valid_0's huber: 3.85943	valid_0's l2: 76.7979	valid_0's l1: 4.66338
SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Bos_taurus', 'Otolemur_garnettii']
Training until validation scores don't improve for 10 rounds
[250]	valid_0's huber: 3.42562	valid_0's l2: 55.6028	valid_0's l1: 4.16304
Did not meet early stopping. Best iteration is:
[250]	valid_0's huber:

MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-24 19:24:58,795] Trial 226 finished with values: [0.8656536549026324, 4.156772369963435, 0.5298702631467377] with parameters: {'boosting_type': 'gbdt', 'lambda_l1': 2.11235684284872, 'lambda_l2': 2.545986499359966, 'max_leaves': 15, 'max_depth': 3, 'feature_fraction': 0.6275052118649216, 'bagging_fraction': 0.7123985676934603, 'learning_rate': 0.03871337664007671, 'min_data_in_leaf': 3, 'drop_rate': 0.1437718204212639}.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported f

0
===== fitting models with seed 0 =====
SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Pan_paniscus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[160]	valid_0's huber: 7.88485	valid_0's l2: 226.76	valid_0's l1: 9.13311
SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[166]	valid_0's huber: 3.45416	valid_0's l2: 64.1343	valid_0's l1: 4.17538
SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Bos_taurus', 'Otolemur_garnettii']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[95]	valid_0's huber: 2.81157	valid_0's l2: 35.7836	valid_0's l1: 3.48468
SEED: 0 | FOLD: 3 | VALIDATION_SPECIES: ['Macaca_nemestrina', 'Microcebus_murinus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[129]	valid_0's huber: 2.52388	va

MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-24 19:27:15,510] Trial 227 finished with values: [0.8844552149543843, 3.915045225713265, 0.47600260628857977] with parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.0276980196726032, 'lambda_l2': 0.544763082731981, 'max_leaves': 20, 'max_depth': 3, 'feature_fraction': 0.345741412241509, 'bagging_fraction': 0.9005959071673288, 'learning_rate': 0.09121990048692946, 'min_data_in_leaf': 3, 'drop_rate': 0.1437718204212639}.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported

...

0
===== fitting models with seed 0 =====
SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Pan_paniscus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[147]	valid_0's huber: 8.86212	valid_0's l2: 307.009	valid_0's l1: 10.2419
SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']
Training until validation scores don't improve for 10 rounds
[250]	valid_0's huber: 3.62523	valid_0's l2: 67.5713	valid_0's l1: 4.39899
Did not meet early stopping. Best iteration is:
[250]	valid_0's huber: 3.62523	valid_0's l2: 67.5713	valid_0's l1: 4.39899
SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Bos_taurus', 'Otolemur_garnettii']
Training until validation scores don't improve for 10 rounds
[250]	valid_0's huber: 3.09036	valid_0's l2: 45.5494	valid_0's l1: 3.79115
Did not meet early stopping. Best iteration is:
[250]	valid_0's huber: 3.09036	valid_0's l2: 45.5494	valid_0's l1: 3.79115
SEED: 0 | FOLD: 3 | VALIDATION_SPEC

MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-24 20:01:54,450] Trial 234 finished with values: [0.8799747707089589, 4.009104624678772, 0.49571000065604176] with parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.3372687152071137, 'lambda_l2': 2.7636630410837917, 'max_leaves': 23, 'max_depth': 3, 'feature_fraction': 0.5809187061009446, 'bagging_fraction': 0.5665557843020324, 'learning_rate': 0.05298955871762984, 'min_data_in_leaf': 8, 'drop_rate': 0.2758291874738865}.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (support

0
===== fitting models with seed 0 =====
SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Pan_paniscus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[121]	valid_0's huber: 9.47398	valid_0's l2: 355.525	valid_0's l1: 10.9245
SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Heterocephalus_glaber', 'Capra_hircus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[150]	valid_0's huber: 3.55081	valid_0's l2: 65.9154	valid_0's l1: 4.32935
SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Bos_taurus', 'Otolemur_garnettii']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[141]	valid_0's huber: 3.5659	valid_0's l2: 62.1976	valid_0's l1: 4.32853
SEED: 0 | FOLD: 3 | VALIDATION_SPECIES: ['Macaca_nemestrina', 'Microcebus_murinus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[152]	valid_0's huber: 2.3601	va

MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-24 20:05:51,339] Trial 235 finished with values: [0.8432968935936161, 4.233353706353105, 0.5263257036244215] with parameters: {'boosting_type': 'gbdt', 'lambda_l1': 2.368954553613788, 'lambda_l2': 1.492191937924149, 'max_leaves': 23, 'max_depth': 3, 'feature_fraction': 0.9825126690946149, 'bagging_fraction': 0.42611385339632235, 'learning_rate': 0.07150591722292106, 'min_data_in_leaf': 3, 'drop_rate': 0.2758291874738865}.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


MultiObjectiveResults(best_trials=[<optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f167349ea90>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f167347b2b0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f167347b4f0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f1673471370>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f1673471610>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f16734716d0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f1673471730>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f1673471790>], all_trials=[<optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f1673429160>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f1670a6b040>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f16734fc550>, <optuna.multi_objective.trial.FrozenMul

In [33]:
best = results.best_trials

In [72]:
for i, t in enumerate(best):
    trait_path = locations.metrics.optimization / trait
    if not trait_path.exists():
        trait_path.mkdir()
    path = trait_path / f"{str(i)}.json"    
    print(f"writing parameters to {path}")
    with open(path, 'w') as f:
        params = t.params
        values = t.values
        to_write = {"number": t.number,"params": params, "metrics": {"R2":values[0], "huber": values[1], "kendall_tau": values[2]}}
        json.dump(to_write, f, sort_keys=True, indent=4)

writing parameters to ../data/metrics/optimization/lifespan/0.json
writing parameters to ../data/metrics/optimization/lifespan/1.json
writing parameters to ../data/metrics/optimization/lifespan/2.json
